In [1]:
from scipy.spatial import distance
from imutils import face_utils
import imutils
import dlib
import mediapipe as mp
import cv2
import datetime
import playsound
import time
import dlib

### dlib Detect Face


- 68 landmarks

![image landmarks](landmarks/landmarks.png) 

In [10]:
detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor('landmarks/shape_predictor_68_face_landmarks.dat')

cap = cv2.VideoCapture(0)
while cap.isOpened():
    ret, frame = cap.read()
    if ret:
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        faces = detector(gray)
        for face in faces:
            pt1 = face.left(), face.top()
            pt2 = face.right(), face.bottom()
            cv2.rectangle(frame, pt1, pt2, (0,255,0), 2)
            landmarks = predictor(gray, face)
            
            for n in range(68):
                x = landmarks.part(n).x
                y = landmarks.part(n).y
                cv2.circle(frame, (x,y), 2, (0,255,0), -1)
    cv2.imshow('Frame', frame)
    key = cv2.waitKey(1)
    if key == 27:
        break

cap.release()
cv2.destroyAllWindows()
                
                

### Face mesh (mediapipe)
- 468 landmarks
![image landmarks](landmarks/facemesh_place.jpg) 

In [11]:
mp_faceMesh= mp.solutions.face_mesh.FaceMesh()

cap = cv2.VideoCapture(0)

while cap.isOpened():
    ret, frame = cap.read()
    if ret:
        rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        height, width, _= frame.shape
        
        res = mp_faceMesh.process(rgb)
        if res.multi_face_landmarks != None:
            for face_landmarks in res.multi_face_landmarks:
                for i in range(468):
                    pt1 = face_landmarks.landmark[i] # (x,y,z)
                    x = int(pt1.x * width)
                    y = int(pt1.y * height)
                    cv2.circle(frame, (x,y) ,2, (0, 255, 0), -1)
                
        cv2.imshow('Cam', frame)
        key = cv2.waitKey(1)
        if key == 27:
            break
cv2.destroyAllWindows()
cap.release()

### Convex Hull

In [17]:
img = cv2.imread('cars.png')
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
_, thresh = cv2.threshold(gray, 50, 255,cv2.THRESH_BINARY)
contours, hierarchy = cv2.findContours(thresh, cv2.RETR_TREE, cv2.CHAIN_APPROX_NONE)

for contour in contours:
    hull = cv2.convexHull(contour)
    cv2.drawContours(img, [hull], -1, (255,0,0),2)
#     cv2.drawContours(img, [contour], -1, (0,255,0),2)
    
cv2.imshow('Image', img)
cv2.imshow('Gray',gray)
cv2.imshow('Threshold',thresh)
cv2.waitKey(0)
cv2.destroyAllWindows()

### Second Project

In [28]:
ALARM_ON = False
MODEL_PATH = "landmarks/shape_predictor_68_face_landmarks.dat"
ALARM_PATH = "sounds/accident.mp3"

def eye_aspect_ratio(eye):
    A = distance.euclidean(eye[1], eye[5])
    B = distance.euclidean(eye[2], eye[4])
    C = distance.euclidean(eye[0], eye[3])
    EAR = (A + B) / (2.0 * C)
    return EAR

def check():
    thresh = 0.25
    frame_check = 10  
    detect = dlib.get_frontal_face_detector()
    predict = dlib.shape_predictor(MODEL_PATH)  

    (lStart, lEnd) = face_utils.FACIAL_LANDMARKS_68_IDXS["left_eye"]
    (rStart, rEnd) = face_utils.FACIAL_LANDMARKS_68_IDXS["right_eye"]
    
    cap = cv2.VideoCapture(0)
    flag = 0
    elapsed_time = 0
    while True:
        ret, frame = cap.read()
        if ret :
            frame = cv2.resize(frame,(640,640))
            gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
            
            faces = detect(gray)
            for face in faces:
                landmarks = predict(gray, face)
                landmarks_face = face_utils.shape_to_np(landmarks)
                leftEye = landmarks_face[lStart:lEnd]
                rightEye = landmarks_face[rStart:rEnd]
                
                leftEAR = eye_aspect_ratio(leftEye)
                rightEAR = eye_aspect_ratio(rightEye)
                EAR = (leftEAR+rightEAR)/2
                
                leftEyeHull = cv2.convexHull(leftEye)
                rightEyeHull = cv2.convexHull(rightEye)
                cv2.drawContours(frame, [leftEyeHull], -1, (0, 255, 0), 1)
                cv2.drawContours(frame, [rightEyeHull], -1, (0, 255, 0), 1)
                
                if thresh> EAR:
                    end_time = time.time()
                    flag += 1
                    
                    if flag > frame_check:
                        if not ALARM_ON:
                            ALARM_ON = True 
                            time.sleep(0.01)
                        
                        cv2.putText(frame, "****************NOOO!****************", (10, 30),
                                cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)
                        elapsed_time = round(end_time - start_time,2)
                        cv2.putText(frame,'sleeping time 00:'+str(elapsed_time)+'sec', (10, 60),
                                    cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)
#                         playsound.playsound(ALARM_PATH, True) 

                else:
                    start_time = time.time()
                    flag = 0
                    ALARM_ON = False
                    
        cv2.putText(frame,'last time :'+str(elapsed_time)+' sec', (400, 620),
                                    cv2.FONT_HERSHEY_COMPLEX, 0.7, (0, 0, 255), 2)
        cv2.imshow("Frame", frame)
        key = cv2.waitKey(1) & 0xFF
        if key == ord("q") or key == 27:
            break
    cap.release()
    cv2.destroyAllWindows()
    
       

In [5]:
check()

SyntaxError: invalid syntax (1850844711.py, line 1)

Number of seconds since January 1, 1970: 1689075577.6898525


Elapsed time (seconds): 2.56
